<a href="https://colab.research.google.com/github/shahd1995913/Machine-Learning/blob/main/6.%20Recommendation%20System%20/Anime_Recommendation_System_by_TAH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [youtub](https://www.youtube.com/watch?v=kccT0FVK6OY&list=PLZoTAELRMXVN7QGpcuN-Vg35Hgjp3htvi&index=3)
## [Github](https://github.com/krishnaik06/Recommendation_complete_tutorial/blob/master/KNN%20Movie%20Recommendation/KNNRecommendation.ipynb)
## [Kaggel](https://www.kaggle.com/competitions/anime-recommendation-system-by-tah/data?select=train.csv)

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download anime-recommendation-system-by-tah
! unzip /content/anime-recommendation-system-by-tah.zip
import pandas as pd

In [ ]:
rating_complete = pd.read_csv("/content/rating_complete.csv")
rating_complete

In [ ]:
# rating_complete
# for i in rating_complete["user_id"]:
#   for x  in rating_complete["rating"]:
#     if x  == 10:
#         print(x + rating_complete["anime_id"])

In [ ]:
rating_complete

In [ ]:
train = pd.read_csv("/content/train.csv")
train

In [ ]:
watching_status = pd.read_csv("/content/watching_status.csv")
watching_status

In [ ]:
anime = pd.read_csv("/content/anime.csv")
anime